# HW4

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Web Scraping

# 1. Define the target URL and use a session (so we can attach cookies for logged-in view)
url = "https://hooslist.virginia.edu/1262/Group/DataScience"
session = requests.Session()

response = session.get(url)

# 2. Parse the HTML
soup = BeautifulSoup(response.content, 'html.parser')
# print(f'soup: {soup}')

# 3. Find all section links — data lives in <a> tags with class js-section-link (data-* attributes)
section_links = soup.find_all('a', class_='js-section-link')
print(f'Found {len(section_links)} section links')

# 4. Extract only fields needed for HW4 (4.2 schema + 4.3 queries):
#   - 4.3.1: subject, course name, meeting time, location, instructor
#   - 4.3.2: course name, meeting time, location, professor
#   - 4.3.3/4.3.4: room + meeting time for conflict detection
#   - 4.3.5: enrollment (salary not on Lou's List; add separately)
rows = []
for link in section_links:
    rows.append({
        'class_number': link.get('data-classnumber'),
        'title': link.get('data-title'),
        'instructors': link.get('data-instructors'),
        'subject': link.get('data-subject'),
        'section': link.get('data-sectioncode'),
        'term': link.get('data-term'),
        'meeting_time': link.get('data-meetings'),  # days, start, end
        'location': link.get('data-location'),
        'enrollment': link.get('data-enrollment'),
    })

print(f'rows: {rows}')

# 5. Load into Pandas
df = pd.DataFrame(rows)
# Drop rows where we didn't get a class number (e.g. header or non-section links)
df = df.dropna(subset=['class_number']).reset_index(drop=True)
print(f'Extracted {len(df)} course sections')
print(df[['class_number', 'title', 'instructors', 'subject', 'section', 'meeting_time', 'location', 'enrollment']].head(10))

Found 169 section links
rows: [{'class_number': '15256', 'title': 'Foundation of Data Science', 'instructors': 'Brian Wright', 'subject': 'DS', 'section': '100', 'term': 'Spring 2026', 'meeting_time': '[{"days":"Mo|We","start":"02:00 PM","end":"03:15 PM"}]', 'location': 'N/A', 'enrollment': '153 / 240 '}, {'class_number': '15257', 'title': 'Foundation of Data Science', 'instructors': 'Ali Rivera', 'subject': 'DS', 'section': '101', 'term': 'Spring 2026', 'meeting_time': '[{"days":"Fr","start":"02:00 PM","end":"02:50 PM"}]', 'location': 'N/A', 'enrollment': '40 / 40 '}, {'class_number': '15296', 'title': 'Foundation of Data Science', 'instructors': 'TBD', 'subject': 'DS', 'section': '102', 'term': 'Spring 2026', 'meeting_time': '[{"days":"Fr","start":"12:00 PM","end":"12:50 PM"}]', 'location': 'N/A', 'enrollment': '27 / 40 '}, {'class_number': '15258', 'title': 'Foundation of Data Science', 'instructors': 'Ali Rivera', 'subject': 'DS', 'section': '103', 'term': 'Spring 2026', 'meeting_t

# 4.1

Confidence: 7

![ERD](hw4.drawio.png)

# 4.2 Data Preparation - Physical Level

## 4.2.1

Confidence: 7

In [12]:
import sqlite3
conn = sqlite3.connect('lous_list.db')  # or ':memory:' for in-memory
conn.execute('PRAGMA foreign_keys = ON')

# Then run your CREATE TABLE statements one by one, e.g.:
conn.execute('''
CREATE TABLE IF NOT EXISTS professors (
    professor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name         TEXT NOT NULL,
    department   TEXT
)
''')
# ... same for rooms, classes, students, enrollments (with AUTOINCREMENT and no ENGINE=InnoDB)
conn.commit()
conn.close()